In [1]:
from manim import *
import pandas as pd
df = pd.read_csv('data.csv')

pd_group_data = df[df['group'].str.startswith('PD')]

display(df)

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [ ]:
import pandas as pd
from manim import *
import numpy as np

# Load and filter data
df = pd.read_csv('data.csv')
pd_group_data = df[df['group'].str.startswith('PD')]
rbd_group_data = df[df['group'].str.startswith('RBD')]
hc_group_data = df[df['group'].str.startswith('HC')]

# Combine all groups into one list for processing
all_groups_data = pd.concat([pd_group_data, rbd_group_data, hc_group_data])

# Define the Manim Scene
class GroupVisualization(Scene):
    def construct(self):
        # Create initial and target lines
        start_line = Line(LEFT * 5, RIGHT * 5, color=WHITE)
        end_line = Line(LEFT * 5 + DOWN * 2, RIGHT * 5 + DOWN * 2, color=WHITE)

        # Create group names at the middle of each line
        pd_text = Text("PD", color=BLUE).scale(0.5).shift(LEFT * 4.5 + UP * 0.5)  # Top-left
        rbd_text = Text("RBD", color=RED).scale(0.5).shift(ORIGIN + UP * 0.5)       # Top-middle
        hc_text = Text("HC", color=GREEN).scale(0.5).shift(RIGHT * 4.5 + UP * 0.5)  # Top-right
        

        # Get the number of all group points
        group_count = len(all_groups_data)
        
        # Create a list to hold dots based on their group
        dots = []

        for i, row in all_groups_data.iterrows():
            # Determine the group and assign color accordingly
            if row['group'].startswith('PD'):
                color = BLUE
            elif row['group'].startswith('RBD'):
                color = RED
            else:
                color = GREEN

            # Calculate position for each point on the start line
            position = interpolate(LEFT * 5, RIGHT * 5, alpha=i / (group_count - 1)) if group_count > 1 else ORIGIN
            dot = Dot(point=position, color=color)
            dots.append(dot)

        # Fade all at once
        self.play(
            FadeIn(start_line), FadeIn(end_line),
            FadeIn(pd_text), FadeIn(rbd_text), FadeIn(hc_text),
            FadeIn(VGroup(*dots)), run_time=1
        )
        self.wait(0.5)

        # Animate points moving to the target line
        animations = []
        for i, (dot, row) in enumerate(zip(dots, all_groups_data.iterrows())):
            _, data_row = row
            rt = data_row['hpr_rt'] / 1000  # Convert reaction time to seconds
            avgv = data_row['hpr_avgv'] / 50  # Normalize average velocity
            target_position = dot.get_center() + DOWN * 2  # Target position on the second line
            distance = np.linalg.norm(target_position - dot.get_center())
            duration = distance / avgv if avgv > 0 else 0  # Calculate time based on average velocity
            
            # Add animation with reaction time delay
            animations.append(
                dot.animate(run_time=duration, rate_func=linear)  # Use linear motion for abrupt movement
                .move_to(target_position)
                .set_delay(rt)
            )
        
        # Play all animations
        self.play(AnimationGroup(*animations))
        
        # Hold the scene
        self.wait(0.5)

# Run the scene
%manim -v WARNING GroupVisualization

Manim Community v0.18.1

In [ ]:
import pandas as pd
from manim import *
import numpy as np

# Load and filter data
df = pd.read_csv('data.csv')
pd_group_data = df[df['group'].str.startswith('PD')]
rbd_group_data = df[df['group'].str.startswith('RBD')]
hc_group_data = df[df['group'].str.startswith('HC')]

# Combine all groups into one list for processing
all_groups_data = pd.concat([pd_group_data, rbd_group_data, hc_group_data])

# Define the Manim Scene
class GroupVisualization(Scene):
    def construct(self):
        # Create initial and target lines
        start_line = Line(LEFT * 5, RIGHT * 5, color=WHITE)
        end_line = Line(LEFT * 5 + DOWN * 2, RIGHT * 5 + DOWN * 2, color=WHITE)

        # Create group names at the middle of each line
        pd_text = Text("PD", color=BLUE).scale(0.5).shift(LEFT * 4.5 + UP * 0.5)  # Top-left
        rbd_text = Text("RBD", color=RED).scale(0.5).shift(ORIGIN + UP * 0.5)       # Top-middle
        hc_text = Text("HC", color=GREEN).scale(0.5).shift(RIGHT * 4.5 + UP * 0.5)  # Top-right
        
        # Fade in everything (lines, text, and dots)
        self.play(
            FadeIn(start_line), FadeIn(end_line),
            FadeIn(pd_text), FadeIn(rbd_text), FadeIn(hc_text)
        )

        # Get the number of all group points
        pd_count = len(pd_group_data)
        rbd_count = len(rbd_group_data)
        hc_count = len(hc_group_data)

        # Define the space between groups
        space_between_groups = 1  # Adjust as needed
        
        # Define the start and end positions for each group on the line
        pd_start_pos = LEFT * 5
        rbd_start_pos = LEFT * 5 + RIGHT * (space_between_groups)
        hc_start_pos = LEFT * 5 + RIGHT * (space_between_groups)

        # Create a list to hold dots based on their group
        dots = []

        # For PD group
        for i, row in pd_group_data.iterrows():
            color = BLUE
            position = interpolate(pd_start_pos, pd_start_pos + RIGHT * pd_count / 10, alpha=i / (pd_count - 1)) if pd_count > 1 else pd_start_pos
            dot = Dot(point=position, color=color)
            dots.append(dot)

        # For RBD group
        for i, row in rbd_group_data.iterrows():
            color = RED
            position = interpolate(rbd_start_pos, rbd_start_pos + RIGHT * rbd_count / 10, alpha=i / (rbd_count - 1)) if rbd_count > 1 else rbd_start_pos
            dot = Dot(point=position, color=color)
            dots.append(dot)

        # For HC group
        for i, row in hc_group_data.iterrows():
            color = GREEN
            position = interpolate(hc_start_pos, hc_start_pos + RIGHT * hc_count / 10, alpha=i / (hc_count - 1)) if hc_count > 1 else hc_start_pos
            dot = Dot(point=position, color=color)
            dots.append(dot)

        # Fade in all dots at once
        self.play(FadeIn(VGroup(*dots)), run_time=1)  # VGroup to group all dots together

        self.wait(0.5)

        # Animate points moving to the target line
        animations = []
        for i, (dot, row) in enumerate(zip(dots, all_groups_data.iterrows())):
            _, data_row = row
            rt = data_row['hpr_rt'] / 1000  # Convert reaction time to seconds
            avgv = data_row['hpr_avgv'] / 100  # Normalize average velocity
            target_position = dot.get_center() + DOWN * 2  # Target position on the second line
            distance = np.linalg.norm(target_position - dot.get_center())
            duration = distance / avgv if avgv > 0 else 0  # Calculate time based on average velocity
            
            # Add animation with reaction time delay
            animations.append(
                dot.animate(run_time=duration, rate_func=linear)  # Use linear motion for abrupt movement
                .move_to(target_position)
                .set_delay(rt)
            )
        
        # Play all animations
        self.play(AnimationGroup(*animations))
        
        # Hold the scene
        self.wait(0.5)

# Run the scene
%manim -v WARNING GroupVisualization


Manim Community v0.18.1